# Facility Location Problem
Pada sesi ini kita akan menyelesaikan facility location problem. Adapun tujuan dari permasalahan ini yaitu untuk meminimalkan biaya instalasi dan alokasi demand dari pusat distribusi ke pelanggan (toko). Data yang digunakan berasal dari mata kuliah Matematika Optimasi yang pernah saya ambil waktu kuliah dengan Bpk Setyo Tri Windras Mara. Saya akan menyelesaikan model ini menggunakan bantuan library pulp.

**Permasalahan:**

Sebuah perusahaan retail berencana melakukan penetrasi pasar ke Kabupaten Kulonprogo karena adanya potensi Bandara NYIA. Perusahaan tersebut hendak membuka sepuluh toko di area Kulonprogo dan saat ini mereka sedang mengevaluasi tiga calon lokasi potensial untuk dibangun pusat distribusi. 

### 1. Mengimpor Library

In [3]:
import pandas as pd
import numpy as np
from pulp import *

### 2. Mengimpor Data
Data yang digunakan memiliki tipe file excel dengan 3 sheet. Oleh karena itu kita gunakan pd.read_excel untuk mengimpor data tersebut. Saya menambahkan beberapa parameter seperti header dan usecols serta melakukan transpose (.T) pada sheet *Biaya* untuk kemudahan akses data.

In [4]:
dist_center = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Pusat Distribusi").set_index("Lokasi potensial")
customers = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Customer").set_index("Customer")
costs = pd.read_excel("Facility Location Problem.xlsx", sheet_name="Biaya", header=1, usecols=range(1,12)).rename(columns={"Unnamed: 1":"Sites"}).set_index("Sites").T

#### a. Data Pusat Distribusi
Tabel ini berisi biaya yang dibutuhkan untuk membangun pusat distribusi serta kapasitas penyimpanan fasilitas tersebut.


In [5]:
dist_center

,Biaya,Kapasitas
Lokasi potensial,,
1,300,250
2,200,150
3,500,300


#### b. Data Pelanggan
Tabel ini berisi 10 toko yang hendak dibuka di Kabupaten Kulonprogo beserta demand per periode 

In [6]:
customers

,Demand per Periode
Customer,
1,30
2,20
3,40
4,50
5,40
6,60
7,30
8,50
9,40


#### c. Data Biaya
Tabel ini berisi biaya yang diperlukan untuk mendistribusikan barang dari pusat distribusi menuju toko

In [7]:
costs

Sites,1,2,3
1,96,76,58
2,97,66,68
3,100,83,100
4,81,52,52
5,89,54,83
6,57,63,74
7,71,96,66
8,96,99,64
9,55,94,63
10,84,86,83


### 3. Menambahkan Variabel
Selain menambahkan variabel, saya juga membuat beberapa parameter yang digunakan yakni:
- list_customer: kumpulan toko 
- list_sites: kumpulan pusat distribusi
- keys: kombinasi toko dan pusat distribusi untuk keperluan pembuatan variabel

Variabel yang digunakan dalam permasalahan ini yaitu:
- yij: alokasi demand dari toko i yang dipenuhi oleh pusat distribusi j. Variabel ini bertipe integer lebih dari 0.
- xij: variabel bertipe binary yang menyatakan kondisi pusat distribusi akan dibangun atau tidak

<img src="https://user-images.githubusercontent.com/61647791/128858577-a3d56c71-ec69-4774-a344-46a860853c7b.PNG" />

*sumber: Sarker and Newton - Optimization Modelling, A Practical Approach*

In [8]:
#create parameter
list_customers = customers.index
list_sites = dist_center.index
keys = [(i,j) for i in list_customers for j in list_sites]

#create variable
y = LpVariable.dicts("y", keys, 0, None, "Integer")
x = LpVariable.dicts("x", list_sites, 0, None, "Binary")

### 4. Menginisiasi Model
Karena tujuan dari permasalahan ini adalah meminimalkan biaya, gunakan **LpMinimize**

In [9]:
model = LpProblem("Facility_Location_Problem", LpMinimize)

### 5. Menambahkan Fungsi Tujuan dan Constrain

**Fungsi Tujuan:**

Tujuan dari permasalahan ini adalah meminimalkan biaya sekaligus biaya yang diperlukan untuk mendistribusikan barang dari pusat distribusi menuju toko

**Batasan:**
 
 - Demand customer/toko (Di) harus dipenuhi
 - Toko i tidak dapat dilayani oleh pusat distribusi j kecuali jika fasilitas tersebut dibangun dan jumlah barang yang disalurkan dari pusat distribusi j ke semua toko harus kurang dari sama dengan kapasitasnya (Uj). 
 
<img src="https://user-images.githubusercontent.com/61647791/128858599-7987d759-af05-4ba0-8295-63b3199e043d.PNG" />

*sumber: Sarker and Newton - Optimization Modelling, A Practical Approach*

In [10]:
#fungsi tujuan
model += lpSum([dist_center.loc[j,"Biaya"]*x[j] for j in list_sites]) +  lpSum([costs.loc[i,j]*y[i,j] for (i,j) in keys])

In [11]:
#constrain
for i in list_customers:
    model += lpSum([y[i,j] for j in list_sites]) == customers.loc[i,"Demand per Periode"]
    
for j in list_sites:
    model += lpSum([y[i,j] - dist_center.loc[j,"Kapasitas"]*x[j] for i in list_customers]) <=0

### 6. Menyelesaikan Model dan Mengecek Hasil Optimasi

In [12]:
#cek status model, jika hasilnya 1 artinya model sudah optimal
model.solve()

1

In [113]:
#cek biaya yang harus dikeluarkan
model.objective.value()

26260

Saya juga membuat tabel yang berisi solusi dari permasalahan di atas supaya lebih mudah dipahami.

In [14]:
solutions = []
for j in list_sites:
    for i in list_customers:
        solutions.append(y[i,j].varValue)

solutions_grouped = [solutions[i:i+10] for i in range(0,len(solutions),10)]
df_sol = pd.DataFrame(solutions_grouped, columns=list_customers, index=list_sites)
df_sol

Customer,1,2,3,4,5,6,7,8,9,10
Lokasi potensial,,,,,,,,,,
1,0,0,0,0,0,60,0,0,40,0
2,0,20,40,50,40,0,0,0,0,0
3,30,0,0,0,0,0,30,50,0,40


In [15]:
#export tabel
df_sol.to_excel("FLP_Solution.xlsx")

Dari tabel di atas, sepuluh toko tersebut mendapatkan pasokan dari tiga pusat distribusi. Tidak ada toko yang tidak mendapatkan bagian. Fasilitas kedua memiliki harga yang lebih murah dengan kapasitas sebesar 150 sehingga terutilisasi sempurna tanpa sisa. Berbeda dengan fasilitas pertama dan ketiga yang masih memiliki sisa kapasitas.

# Penutup
Demikian tutorial singkat menyelesaikan permasalahan facility location problem menggunakan pulp, mohon maaf jika terdapat salah maupun kekurangan. Selamat mencoba :)